Legenda:

n_steps = número de passos da receita

minutes = tempo de preparo em minutos

nutrition = Valor de calorias e valores diários de gordura total, açucar, sódio, proteína, gordura saturada e carboidratos

id = identificação da receita

recipe_id = identificação da receita

user_id = identificação do usuário

contributor_id = identificação de quem contribuiu com a receita

date = data da interação

rating = nota dada

name = nome da receita

submitted = data que a receita foi cadastrada

review = comentário

steps = passos da receita

tags = rótulos da receita



In [22]:
#Importando as bibliotecas
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
RAW_recipes = pd.read_csv("https://media.githubusercontent.com/media/PedroCosDi/ProjetoAplicadoMack3/main/Dataset/RAW_recipes.csv")
RAW_interactions = pd.read_csv("https://media.githubusercontent.com/media/PedroCosDi/ProjetoAplicadoMack3/main/Dataset/RAW_interactions.csv")


In [23]:
# Mostra informações resumidas dos DataFrames
print("Informações Resumidas para o DataFrame RAW_recipes:")
print(RAW_recipes.info())
print(RAW_recipes.describe())
print("\nInformações Resumidas para o DataFrame RAW_interactions:")
print(RAW_interactions.info())
print(RAW_interactions.describe())

Informações Resumidas para o DataFrame RAW_recipes:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231637 non-null  int64 
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 21.2+ MB
None
                  id       minutes  contributor_id        n_steps  \
count  231637.000000  2.316370e+05    2.316370e+05  231637.000000   
mean   222014

In [24]:
# Acima descobrimos que a coluna minutes de RAW_recipes está cheia de erros, com valores extremamente altos, por isso limitamos ela a até 60 minutos

# Seleciona só os valores com 60 ou menos na coluna minutes
RAW_recipes = RAW_recipes[RAW_recipes['minutes'] <= 60]

# Também vimos que o n_steps de RAW_recipes tem um grande salto entre 75% e seu max, então vamos tirar os outliers

# Calcula o IQR da coluna 'n_steps'
Q1 = RAW_recipes['n_steps'].quantile(0.25)
Q3 = RAW_recipes['n_steps'].quantile(0.75)
IQR = Q3 - Q1

# Define os limites inferior e superior para identificar outliers
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Remove os outliers da coluna 'n_steps'
RAW_recipes = RAW_recipes[RAW_recipes['n_steps'].between(limite_inferior, limite_superior)]

print(RAW_recipes.describe())


                  id        minutes  contributor_id        n_steps  \
count  164063.000000  164063.000000    1.640630e+05  164063.000000   
mean   223732.118985      29.118771    4.883149e+06       8.399901   
std    140905.379819      16.208558    9.317623e+07       4.173869   
min        40.000000       0.000000    2.700000e+01       1.000000   
25%    101801.000000      15.000000    5.704200e+04       5.000000   
50%    211115.000000      30.000000    1.784270e+05       8.000000   
75%    335523.500000      40.000000    4.084540e+05      11.000000   
max    537716.000000      60.000000    2.002285e+09      20.000000   

       n_ingredients  
count  164063.000000  
mean        8.472843  
std         3.413810  
min         1.000000  
25%         6.000000  
50%         8.000000  
75%        11.000000  
max        38.000000  


In [25]:
# Mostra a quantidade de valores unicos em cada Dataset

# Para RAW_recipes
print("Dataset: RAW_recipes")
for coluna in RAW_recipes.columns:
    num_valores_unicos = RAW_recipes[coluna].nunique()
    print(f"Coluna '{coluna}': {num_valores_unicos} valores exclusivos")

# Para RAW_interactions
print("\nDataset: RAW_interactions")
for coluna in RAW_interactions.columns:
    num_valores_unicos = RAW_interactions[coluna].nunique()
    print(f"Coluna '{coluna}': {num_valores_unicos} valores exclusivos")

Dataset: RAW_recipes
Coluna 'name': 163257 valores exclusivos
Coluna 'id': 164063 valores exclusivos
Coluna 'minutes': 61 valores exclusivos
Coluna 'contributor_id': 22866 valores exclusivos
Coluna 'submitted': 4898 valores exclusivos
Coluna 'tags': 147584 valores exclusivos
Coluna 'nutrition': 162166 valores exclusivos
Coluna 'n_steps': 20 valores exclusivos
Coluna 'steps': 163582 valores exclusivos
Coluna 'description': 157673 valores exclusivos
Coluna 'ingredients': 163247 valores exclusivos
Coluna 'n_ingredients': 33 valores exclusivos

Dataset: RAW_interactions
Coluna 'user_id': 226570 valores exclusivos
Coluna 'recipe_id': 231637 valores exclusivos
Coluna 'date': 6396 valores exclusivos
Coluna 'rating': 6 valores exclusivos
Coluna 'review': 1125282 valores exclusivos


In [26]:
# Verifica valores ausentes em RAW_recipes
print("Dataset: RAW_recipes")
print(RAW_recipes.isna().sum())

# Verifica valores ausentes em RAW_interactions
print("\nDataset: RAW_interactions")
print(RAW_interactions.isna().sum())

Dataset: RAW_recipes
name                 1
id                   0
minutes              0
contributor_id       0
submitted            0
tags                 0
nutrition            0
n_steps              0
steps                0
description       3703
ingredients          0
n_ingredients        0
dtype: int64

Dataset: RAW_interactions
user_id        0
recipe_id      0
date           0
rating         0
review       169
dtype: int64


In [27]:
# Transformando objeto em date
RAW_recipes['submitted'] = pd.to_datetime(RAW_recipes['submitted'])

# Encontre a primeira e a ultima data de cadastro
menor_data = RAW_recipes1['submitted'].min()
maior_data = RAW_recipes1['submitted'].max()

print("Menor Data:", menor_data)
print("Maior Data:", maior_data)

Menor Data: 1999-08-06 00:00:00
Maior Data: 2018-12-04 00:00:00


In [28]:
# Remove as colunas 'contributor_id' e 'id'
colunas_a_remover = ['contributor_id', 'id']
RAW_recipes1 = RAW_recipes.loc[:, ~RAW_recipes.columns.isin(colunas_a_remover)]

# Cria gráficos de barras para as colunas de texto (strings)
colunas_de_texto = RAW_recipes1.select_dtypes(include=['int64']).columns

for coluna in colunas_de_texto:
    fig = px.bar(RAW_recipes1[coluna].value_counts().reset_index(), x='index', y=coluna, title=f'Distribuição de {coluna}')
    fig.update_xaxes(title=coluna)
    fig.update_yaxes(title='Contagem')
    fig.show()

In [29]:
# Cria gráfico de barras para a coluna 'rating'
fig = px.bar(RAW_interactions['rating'].value_counts().reset_index(), x='index', y='rating')
fig.update_xaxes(title='Rating')
fig.update_yaxes(title='Contagem')
fig.update_layout(title='Distribuição da Coluna "rating"')
fig.show()

In [30]:
# Seleciona apenas as colunas numéricas relevantes do dataset RAW_recipes
numeric_columns = ['minutes', 'n_steps', 'n_ingredients']

# Mescla os datasets usando a coluna 'id' como chave
merged_data = pd.merge(RAW_recipes[numeric_columns], RAW_interactions['rating'], left_index=True, right_index=True, how='inner')

# Calcula a correlação de Pearson
correlation_matrix = merged_data.corr()

# Exibe a matriz de correlação
print("Matriz de Correlação:")
print(correlation_matrix)


Matriz de Correlação:
                minutes   n_steps  n_ingredients    rating
minutes        1.000000  0.439358       0.378338  0.003146
n_steps        0.439358  1.000000       0.414642 -0.000102
n_ingredients  0.378338  0.414642       1.000000  0.001304
rating         0.003146 -0.000102       0.001304  1.000000
